# PULP przykład #2

https://hackernoon.com/linear-programming-in-python-a-straight-forward-tutorial-a0d152618121

Żeby mieć pomysł, rozwiążemy prosty problem dotyczący harmonogramowania produkcji. Wyobraź sobie, że pracujesz dla firmy, która buduje komputery. Komputer jest dość złożonym produktem i jest kilka fabryk, które je montują, a firma płaci określoną kwotę za sztukę. Koszt tego modelu komputera na rynku jest ustalony na 500 $, różne fabryki montują komputery z różnymi prędkościami i kosztami. Fabryka: 

-  f0 produkuje 2000 dziennie po 450 $ za sztukę 

-  f1 1500 dziennie po 420 $ za sztukę

-  f2 1000 dziennie po 400 $ za sztukę 

Mamy 1 miesiąc na zmontowanie 80 000 jednostek pod warunkiem, że żadna fabryka nie będzie produkować więcej niż dwa razy więcej jednostek niż jakakolwiek inna fabryka. 

Pytanie brzmi,jaka jest optymalna alokacja produkcji między fabrykami, tak aby zmaksymalizować zysk uzyskany ze sprzedaży komputerów przy tych ograniczeniach?

 aby rozwiązać problem produkcji komputerowej za pomocą programowania liniowego, potrzebujemy następujących rzeczy:

- Zbiór zmiennych decyzyjnych
- Zestaw więzów liniowych dotyczących tych zmiennych
- Liniowa funkcja celu maksymalizująca lub minimalizująca


In [1]:
from pulp import *
# cel maksymalizacja zysku ze sprzedaży określonej liczby komputerów

problem = LpProblem("problemName", LpMaximize)

Definiujemy również stałe, które otrzymaliśmy z opisu problemu:

In [2]:
# factory cost per day
cf0 = 450
cf1 = 420
cf2 = 400

In [3]:
# przepustowość fabryki na dzień 
f0 = 2000 
f1 = 1500 
f2 = 1000

In [4]:
# cel produkcji
goal = 80000

In [5]:
# time limit
max_num_days = 30

In [6]:
num_factories = 3

## Definiuje zmienne decyzje
robi się to tak:
### - variable = LpVariable("variableName")

Jeszcze innym użytecznym sposobem definiowania zmiennych w PuLP jest użycie funkcji dicts. Może to być bardzo przydatne w przypadkach, gdy musimy zdefiniować dużą liczbę zmiennych tego samego typu i granic, zmiennaNames byłaby listą kluczy do słownika:

### - varDict = LpVariable.dicts ("varDict", variableNames, lowBound, upBound)

 ##  <span style="color:blue"> Definiujemy zmienną decyzyjną
    
   Celem jest pokazanie ile dni ma produkować każda fabryka

    x0  - ilość dni pracy fabryki 0
    x1  - ilość dni pracy fabryki 1
    x2  - ilość dni pracy fabryki 2
    
 mamy przekazać zarządowi firmy, ile dni każda z 3 fabryk ma produkować
 
 Zmiennymi decyzyjnymi dla problemu produkcji komputerów są dni, które spędzamy na produkcji dla każdej fabryki:

In [20]:
# factories
num_factories = 3
factory_days = LpVariable.dicts("factoryDays", list(range(num_factories)), lowBound=0, upBound=30, cat="Continuous")
factory_days

{0: factoryDays_0, 1: factoryDays_1, 2: factoryDays_2}

In [37]:
## Tabela ograniczeń

from beautifultable import BeautifulTable

table = BeautifulTable()
table.column_headers = ["Maszyna", "koszt produkcji","Maksymalna ilość produkcji w miesiącu"]
table.append_row(["Fabryka f0", 450, 2000])
table.append_row(["Fabryka f1", 420, 1500])
table.append_row(["Fabryka f2", 400, 1000])
print(table)

+------------+-----------------+---------------------------------------+
|  Maszyna   | koszt produkcji | Maksymalna ilość produkcji w miesiącu |
+------------+-----------------+---------------------------------------+
| Fabryka f0 |       450       |                 2000                  |
+------------+-----------------+---------------------------------------+
| Fabryka f1 |       420       |                 1500                  |
+------------+-----------------+---------------------------------------+
| Fabryka f2 |       400       |                 1000                  |
+------------+-----------------+---------------------------------------+


##  <span style="color:blue">  ograniczenie celu
    że łaczna produkcja we wszystkich fabrykach nie może być wieksza niż 80000 komputerów
    
#### przepustowość fabryki na dzień 
- f0 = 2000 
- f1 = 1500 
- f2 = 1000

- goal = 80000

\begin{align} 
{2000x_0+1500x_1+1000x_2}  & \leqslant 80000 \\ 
\end{align} 

In [22]:
c1 = factory_days[0]*f0 + factory_days[1]*f1 + factory_days[2] * f2 >= goal

##  <span style="color:blue"> Ograniczenia produkcyjne

Należy zwrócić uwagę, że przy programowaniu liniowym ograniczenia muszą być liniowe. Ograniczenia, na których nam zależy, polegają na tym, aby liczba montowanych jednostek była większa lub równa docelowej ilości oraz ograniczenie produkcyjne, że żadna fabryka nie powinna produkować więcej niż dwa razy więcej niż inna fabryka:

factory_days to liczba dni pracy fabryki, zdefiniowane jako liczba całkowita z zakresu od 1 do 30 


In [23]:
# production constraints
c2 = factory_days[0]*f0 <= 2*factory_days[1]*f1   # liczba produkcji fabryki 0 <= produkcja fabryki 1
c3 = factory_days[0]*f0 <= 2*factory_days[2]*f2   # liczba produkcji fabryki 0 <= produkcja fabryki 2
c4 = factory_days[1]*f1 <= 2*factory_days[2]*f2   # liczba produkcji fabryki 1 <= produkcja fabryki 0
c5 = factory_days[1]*f1 <= 2*factory_days[0]*f0   # liczba produkcji fabryki 1 <= produkcja fabryki 2
c6 = factory_days[2]*f2 <= 2*factory_days[1]*f1
c7 = factory_days[2]*f2 <= 2*factory_days[0]*f0

##  <span style="color:blue">Zebraliśmy 7 ograniczeń, teraz dodajemy je do problemu
    
    to jest nasz zdefiniowany wcześiej problem
###    problem = LpProblem("problemName", LpMaximize)

In [24]:
# adding the constraints to the problem
problem += c1
problem += c2
problem += c3
problem += c4
problem += c5
problem += c6
problem += c7

##  <span style="color:blue"> Funkcja celu

Celem problemu montażu komputerów jest zasadniczo minimalizacja kosztów montażu wszystkich tych komputerów. Można to po prostu zapisać jako maksymalizację kosztu ujemnego:

### <span style="color:green">       ilość dni pracy * wydajność fabryki * koszty dzienne produkcji

dodaje do problemu funkcję celu

In [26]:
# objective function
problem += -factory_days[0]*cf0*f0 - factory_days[1]*cf1*f1 - factory_days[2]*cf2*f2

Zebaliśmy do modelu dużo warunków ograniczających i funkcje celu. teraz można to wydrukować.

In [27]:
print(problem)

problemName:
MAXIMIZE
-900000*factoryDays_0 + -630000*factoryDays_1 + -400000*factoryDays_2 + 0
SUBJECT TO
_C1: 2000 factoryDays_0 + 1500 factoryDays_1 + 1000 factoryDays_2 >= 80000

_C2: 2000 factoryDays_0 - 3000 factoryDays_1 <= 0

_C3: 2000 factoryDays_0 - 2000 factoryDays_2 <= 0

_C4: 1500 factoryDays_1 - 2000 factoryDays_2 <= 0

_C5: - 4000 factoryDays_0 + 1500 factoryDays_1 <= 0

_C6: - 3000 factoryDays_1 + 1000 factoryDays_2 <= 0

_C7: - 4000 factoryDays_0 + 1000 factoryDays_2 <= 0

VARIABLES
factoryDays_0 <= 30 Continuous
factoryDays_1 <= 30 Continuous
factoryDays_2 <= 30 Continuous



## Rozwiązywanie
Po zdefiniowaniu wszystkiego, co niezbędne w naszym zadaniu programowania liniowego, możemy wywołać metodę rozwiązywania, która daje:
   -  1, jeśli problem został rozwiązany, 
   - -1, jeśli było niewykonalne

In [29]:
problem.solve()

1

In [31]:
for i in range(3):
    print(f"Factory {i}: {factory_days[i].varValue}")

Factory 0: 8.3333333
Factory 1: 22.222222
Factory 2: 30.0


W programowaniu liniowym zakładamy, że relacje między zmiennymi są liniowe, a same zmienne są ciągłe. W ramach kontynuacji tego samouczka omówię programowanie liczb całkowitych mieszanych, w którym zmienne mogą być liczbami całkowitymi, co okaże się bardzo użyteczną rzeczą, ponieważ można je wykorzystać do symulacji logiki boolowskiej.